TODO:

popsat, že cviko se také zaměřuje na práci se světly a filtry

uvést glosář pojmů k line scan kamerám

# Čtení QR kódů řádkovou kamerou
Cvičení je zaměřené na práci s řádkovými kamerami, které díky svým specifickým vlastnostem budeme využívat pro zisk obrazů skla, na kterých budeme detekovat škrábance pomocí metod zpracování obrazu.

## Řádkové kamery
Řádková kamera neboli řádkový skener je speciální druh kamery, která snímá pouze 1 řádek. Díky tomu je schopna dosáhnout obrovských frekvencí snímání (až ticíce řádků za 1 sekundu). V labu máme kamery [Basler Racer](https://www.baslerweb.com/en/products/cameras/line-scan-cameras/racer/) o velikosti řádku 6k, 8k a 12k pixelů.

<img src="images/racer.png" width="25%">

Z podstaty skeneru však vyplývá, že se objekt pod kamerou musí pohybovat. V případě, že se objekt nebude pohybovat, vytvoří kamera obrázek, který bude mít všechny řádky totožné (pouze ten 1, který kamera snímá). 

Tím, že je kamera schopná dosahovat neskutečných frekvencí snímání dále umožňuje získávat obrazová data v neskutečně velkém rozlišení. Na druhou stranu nás tím nutí nastavovat nízkou dobu expozice a tedy je zapotřebí mnohem více světla.

Velikost obrázků z kamery je variabilní. Šířka obrázku odpovídá velikosti řádku a počet snímaných řádků lze v kameře nastavit. Kamera je tedy schopna poskytnout obrázek tak, jak ho známe (matici), ale až potom, co si nasnímá všechny řádky. 

---

### Import knihoven a konfigurace

In [1]:
%run ../svz.ipynb

### Pomocné funkce
Z následujících funkcí je potřeba vybírat ty vhodné pro splnění úkolu.

Seznam funkcí pro přehlednost:
- [`connect_camera(...)`](../svz.ipynb#connect_functions)


- [`show_images(...)`](../svz.ipynb#show_functions)
- [`plot_images(...)`](../svz.ipynb#show_functions)


- [`to_gray(...)`](../svz.ipynb#preprocessing_functions)
- [`to_hsv(...)`](../svz.ipynb#preprocessing_functions)
- [`negative(...)`](../svz.ipynb#preprocessing_functions)
- [`crop(...)`](../svz.ipynb#preprocessing_functions)
- [`crop_by_bounding_rect(...)`](../svz.ipynb#preprocessing_functions)
- [`crop_contour(...)`](../svz.ipynb#preprocessing_functions)
- [`contour_to_image(...)`](../svz.ipynb#preprocessing_functions)
- [`resize(...)`](../svz.ipynb#preprocessing_functions)


- [`filtration_median(...)`](../svz.ipynb#filtration_functions)
- [`fill_holes(...)`](../svz.ipynb#filtration_functions)


- [`to_intensity(...)`](../svz.ipynb#others)
- [`logical_and(...)`](../svz.ipynb#others)

---

### Úkol 1
Seznamte se s řádkovou kamerou a jejími parametry, jejichž nastavení je pro správné snímání kritické. Vyzkoušejte si ostření řádkové kamery. Nakonec se seznamte s vlastní rychlostí snímání kamery a získejte obrázek skla, na kterém detekujte škrábance ve tvaru příslušného čísla.

<div style="color: blue; text-align: right">[ 1 bod ]</div>

#### 1) Seznamte se s řádkovou kamerou
Připojte řádkovou kameru a otevřete ji v **Pylonu**. Nastavte si **vlastní ID** kamery pro lepší identifikaci kamery v síti. Seznamte se s nastavením následujících parametrů, které si uložte do oblíbených ("Favourites").

- Exposure time (Abs)
- Width, Height
- Gain
- Gamma
- Acquisition Frame Count

#### 2) Zaostřete řádkovou kameru
Zaostřete kameru, využijte k tomu **mřížkový podklad** držáků Kaiser. Správně zaostřenou kameru poznáte podle toho, že budou vidět vedle sebe svislé čáry různých intenzit.

Nezapomeňte **přidat osvětlení**, které bude svítit na místo řádku, na které kamera vidí. 

#### 3) Získejte obrázek černobílého QR kódu
Využijte manuál kamery jako zdroj textu, který chcete snímat. Zkoordinujte správně rychlost pohybu textu pod kamerou a zisk snímku tak, abyste získali text. Pozor ať není text příliš roztažený či zploštělý.

In [123]:
image = load_image('5.png') ###
show_images(image)

#### 5) Upravte obrázek do podoby, v jaké půjde QR kód přečíst
TODO text

In [134]:
height, width, channels = image.shape
width, height

(6144, 3025)

In [135]:
new_width, new_height = width // 5, height // 11 ###
resized_image = resize(image, (new_width, new_height)) ###
show_images(resized_image)

#### 6) Pomocí knihovy pyzbar přečtěte z QR kódu zakódovaný text a získejte souřadnice bounding boxu

In [136]:
import pyzbar

In [158]:
decoded = pyzbar.pyzbar.decode(resized_image)
if len(decoded) < 1:
    raise RuntimeError("Nebyl nalezen zadny QR kod")
qr = decoded[0]
qr_data, qr_rect = qr.data, qr.rect
qr_data, qr_rect

(b'Kazdy Fitak chce energetak!', Rect(left=540, top=69, width=88, height=90))

In [153]:
qr_text = qr_data.decode("utf-8")
qr_text

'Kazdy Fitak chce energetak!'

#### 7) Do původního obrázku vykreslete ohraničující obdélník a vypište získaný text

In [154]:
rect.left, rect.top, rect.width, rect.height

(540, 69, 88, 90)

In [155]:
top_left_corner_x = rect.left
top_left_corner_y = rect.top
bottom_right_x = rect.left + rect.width
bottom_right_y = rect.top + rect.height

In [156]:
annotated_image = resized_image.copy()
cv2.rectangle(annotated_image, (top_left_corner_x, top_left_corner_y), (bottom_right_x, bottom_right_y), (0, 255, 0), 3)
cv2.putText(annotated_image, qr_text, (top_left_corner_x, top_left_corner_y - 20), 0, 0.5, (0, 255, 0), 2)
show_images(annotated_image)

---

### Úkol 2
Seznamte se s řádkovou kamerou a jejími parametry, jejichž nastavení je pro správné snímání kritické. Vyzkoušejte si ostření řádkové kamery. Nakonec se seznamte s vlastní rychlostí snímání kamery a získejte obrázek skla, na kterém detekujte škrábance ve tvaru příslušného čísla.

<div style="color: blue; text-align: right">[ 1 bod ]</div>

#### 1) Získejte obrázek černobílého QR kódu
Využijte manuál kamery jako zdroj textu, který chcete snímat. Zkoordinujte správně rychlost pohybu textu pod kamerou a zisk snímku tak, abyste získali text. Pozor ať není text příliš roztažený či zploštělý.

In [160]:
image = load_image('5.png') ###
show_images(image)

#### 2) Upravte obrázek do podoby, v jaké půjde QR kód přečíst
TODO text

In [161]:
height, width, channels = image.shape
width, height

(6144, 3025)

In [168]:
new_width, new_height = width // 5, height // 11 ###
resized_image = resize(image, (new_width, new_height)) ###
show_images(resized_image)

In [169]:
negative_image = negative(resized_image)
show_images(negative_image)

#### 3) Pomocí knihovy pyzbar přečtěte z QR kódu zakódovaný text

In [163]:
decoded = pyzbar.pyzbar.decode(resized_image)
if len(decoded) < 1:
    raise RuntimeError("Nebyl nalezen zadny QR kod")
qr = decoded[0]
qr_data, qr_rect = qr.data, qr.rect
qr_data, qr_rect

(b'Kazdy Fitak chce energetak!', Rect(left=540, top=69, width=88, height=90))

In [164]:
qr_text = qr_data.decode("utf-8")
qr_text

'Kazdy Fitak chce energetak!'